<a id="top"></a>
# ULLYSES Data Walkthrough Notebook

***

## Learning Goals

By the end of this tutorial, you will:

- Know the different types of high level science products (HLSPs) offered by the ULLYSES team.
- Be able to open each file and view its contents and metadata.
- Understand how to use each type of HSLP.

## Table of Contents
**0. [Introduction](#introduction)**

**1. [HLSP Descriptions and Filename Convention](#hlspdescrip)**

**2. [HLSP File Exploration](#hlspformats)**

\- 2.1 [Non-Variable Spectra](#nonvariable)

\- 2.2 [Time-series Spectra](#tss)

\- 2.3 [WFC3 Drizzled Images](#drizzle)

<a id="introduction"></a>
## Introduction

The Hubble Space Telescope’s (HST) Ultraviolet Legacy Library of Young Stars as Essential Standards ([ULLYSES](https://ullyses.stsci.edu/index.html)) program has devoted approximately 1,000 HST orbits to the production of an ultraviolet spectroscopic library of young high- and low-mass stars in the local universe. This Director’s Discretionary program was designed to take advantage of HST’s unique UV capabilities, as both high- and low-mass stars feature different complex UV emission processes that strongly impact their surroundings, but are difficult to model. The UV emission from star formation is central to a wide range of vital astrophysical issues, ranging from cosmic reionization to the formation of planets.

The ULLYSES program uniformly sampled the fundamental astrophysical parameter space for each mass regime — including spectral type, luminosity class, and metallicity for massive OB stars (in the Magellanic Clouds and two other lower-metallicity nearby galaxies) and the mass, age, and disk accretion rate for low-mass T Tauri stars (in eight young Galactic associations). The data were gathered over a three-year period, from Cycle 27 through Cycle 29 (2020-2022). The design and targets of these observations were determined in partnership with the astronomical community, allowing researchers from around the world to help develop the final program and to plan coordinated observations with other space- and ground-based telescopes.

The ULLYSES team produces several types of High Level Science Products (HLSPs), which we will walk through in this notebook. Products are made using both archival data and new HST observations obtained through the ULLYSES program. Data products are available from the [ULLYSES search catalog](https://mast.stsci.edu/search/ui/#/ullyses) (HLSPs only), the [MAST Data Discovery Portal](https://mast.stsci.edu/) (HLSPs and contributing data), or directly as a High-Level Science Product collection using the [DOI](https://archive.stsci.edu/hlsp/ullyses) (HLSPs only). More information about ULLYSES data products can be found under the ["Data"](https://ullyses.stsci.edu/ullyses-download.html) tab on the ULLYSES website.

### Imports
- `numpy` to handle array functions
- `astropy.io` : `fits` for accessing FITS files
- `astroy.wcs` : `WCS` to create world coordinate system objects for plotting WFC3 images
- `astropy.table` : `Table` for creating tidy tables of the data
- `matplotlib.pyplot` for plotting data
- *%matplotlib widget* enables interactive plotting - install using `pip install ipympl`
- `pathlib` : `Path` to create product and data directories
- `shutil` to perform directory and file operations
- `os` to interact with the operating system
- `glob` to find all files in the directory
- `ullyses_utils` : `match_aliases` to use the target name in the filename of the ULLYSES HLSP

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from astropy.time import Time
import matplotlib.pyplot as plt
import os
from pathlib import Path
import shutil
import glob

from ullyses_utils.match_aliases import match_aliases

%matplotlib widget
plt.rcParams['figure.figsize'] = 10, 6
plt.style.use('seaborn-v0_8-colorblind')

In [ ]:
# Set up a quick function to bin the spectra
def downsample_1d(myarr,factor):
    """
    Downsample a 1D array by averaging over <factor> pixels.
    Crops right side if the shape is not a multiple of factor.
    Got this specific function from "Adam Ginsburg's python codes" on agpy

    myarr : numpy array

    factor : how much you want to rebin the array by
    """
    xs = myarr.shape[0]
    crarr = myarr[:xs-(xs % int(factor))]
    dsarr = np.mean(np.concatenate(
                     [[crarr[i::factor] for i in range(factor)]]
                     ), axis=0)

    return dsarr

***

<a id="hlspdescrip"></a>
## HLSP Descriptions and Filename Convention

The file names for ULLYSES science data products follow a specific naming scheme. This includes the target name, instrument(s), and observing configuration(s) that contributed to the product. However, not all products will appear in the early releases. File names have the form:

`hlsp_ullyses_<telescope>_<instrument>_<target>_<opt_elem>_<version>_<product-type>`

where
`<target>` is the target name and `<version>` is the data release identifier (dr1, dr2, etc.)

The `<telescope>`, `<instrument>`, `<opt_elem>`, and `<product-type>` templates take names from the following table:

| Description                                                                                                  | Telescope | Instrument    | Opt-Elem                                        | Product-Type      | Level |
|--------------------------------------------------------------------------------------------------------------|-----------|---------------|-------------------------------------------------|-------------------|-------|
| Custom calibrated STIS 1D spectra                                                                            | hst       | stis          | g230l, g430l, or g750l                             | spec.fits         | 0     |
| STIS custom calibration parameter files                                                                      | hst       | stis          | g230l, g430l, or g750l                             | spec.yaml         | 0     |
| Combined spectra, with common instrument and  grating, and in some cases with different cenwave settings.    | hst       | cos           | g130m, g160m, g185m, or g230l                      | cpsec.fits        | 2     |
|                                                                                                              |           | stis          | e140l, e230h, e140m, e230m, g230l, g430l, or g750l |                   |       |
| Combined spectra, with common instrument, different gratings and cenwave settings, and grouped by resolution | fuse      | fuv           | lwrs or mdrs                                    | aspec.fits        | 3     |
|                                                                                                              | hst       | cos           | g130m-g160m-g185m                               |                   |       |
|                                                                                                              |           | stis          | e140h-e230h, g140m-e230m or g230l-g430l-g750l |                   |       |
| All instruments and settings abutted together*                                                               | hst       | cos-stis      | uv or uv-opt                                    | preview-spec.fits | 4     |
|                                                                                                              | hst-fuse  | fuse-cos-stis | uv-opt                                          |                   |       |
| Exposure-level time-series spectra                                                                           | hst       | cos           | g130m or g160m                                    | tss.fits          | 5     |
|                                                                                                              | lcogt     | 04m           | v-ip                                            |                   |       |
| Subexposure-level time-series spectra                                                                        | hst       | cos           | g130m or g160m                                    | split-tss.fits    | 5     |
| WFC3 drizzled images                                                                                         | hst       | wfc3          | f225w, f275w, f336w, f475w or f814w               | drc.fits          | 6     |

***

<a id="hlspformats"></a>
## HLSP File Formats

Most High Level Science Products are in FITS format. The organization of each FITS file depends on the HLSP type. There are three broad categories of HLSPs: [non-variable spectra](#nonvariable), [time-series spectra](#tss), and [drizzled images](#drizzle).

<a id="nonvariable"></a>
### Non-variable Spectra

#### FITS File Structure

Spectral data and information is stored in two BINTABLE extensions:
- a Science extension for the combined product, and
- a matching Provenance extension to record attributes of each spectrum that contributed to the combined product

Let's start by downloading the available data products for a target with no known variability and look around. More information about the different ULLYSES data product levels can be found on the [ULLYSES website](https://ullyses.stsci.edu/ullyses-data-description.html#productDescrip). You are also able to directly download the data from the [ULLYSES search form](https://mast.stsci.edu/search/ui/#/ullyses/) hosted on MAST.

We will download one example star from the high mass sample (SK -67 167) and one example star from the low mass sample (XX Cha).

In [ ]:
# Use curl to download the files -- these are the level 2, 3, and 4 data products for the target SK -67 167
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fsk-67d167%2Fdr7%2Fhlsp_ullyses_fuse_fuv_sk-67d167_lwrs_dr7_aspec.fits" --output "hlsp_ullyses_fuse_fuv_sk-67d167_lwrs_dr7_aspec.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fsk-67d167%2Fdr7%2Fhlsp_ullyses_hst-fuse_fuse-stis_sk-67d167_uv_dr7_preview-spec.fits" --output "hlsp_ullyses_hst-fuse_fuse-stis_sk-67d167_uv_dr7_preview-spec.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fsk-67d167%2Fdr7%2Fhlsp_ullyses_hst_stis_sk-67d167_e140m_dr7_cspec.fits" --output "hlsp_ullyses_hst_stis_sk-67d167_e140m_dr7_cspec.fits" --fail --create-dirs

In [ ]:
# Use curl to download the files -- these are the level 2, 3, and 4 data products for the target XX Cha
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fv-xx-cha%2Fdr7%2Fhlsp_ullyses_hst_cos-stis_v-xx-cha_uv-opt_dr7_preview-spec.fits" --output "hlsp_ullyses_hst_cos-stis_v-xx-cha_uv-opt_dr7_preview-spec.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fv-xx-cha%2Fdr7%2Fhlsp_ullyses_hst_stis_v-xx-cha_g230l_dr7_cspec.fits" --output "hlsp_ullyses_hst_stis_v-xx-cha_g230l_dr7_cspec.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fv-xx-cha%2Fdr7%2Fhlsp_ullyses_hst_stis_v-xx-cha_g430l_dr7_cspec.fits" --output "hlsp_ullyses_hst_stis_v-xx-cha_g430l_dr7_cspec.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fv-xx-cha%2Fdr7%2Fhlsp_ullyses_hst_cos_v-xx-cha_g160m_dr7_cspec.fits" --output "hlsp_ullyses_hst_cos_v-xx-cha_g160m_dr7_cspec.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fv-xx-cha%2Fdr7%2Fhlsp_ullyses_hst_stis_v-xx-cha_g750l_dr7_cspec.fits" --output "hlsp_ullyses_hst_stis_v-xx-cha_g750l_dr7_cspec.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fv-xx-cha%2Fdr7%2Fhlsp_ullyses_hst_cos_v-xx-cha_g130m_dr7_cspec.fits" --output "hlsp_ullyses_hst_cos_v-xx-cha_g130m_dr7_cspec.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fv-xx-cha%2Fdr7%2Fhlsp_ullyses_hst_cos_v-xx-cha_g130m-g160m_dr7_aspec.fits" --output "hlsp_ullyses_hst_cos_v-xx-cha_g130m-g160m_dr7_aspec.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fv-xx-cha%2Fdr7%2Fhlsp_ullyses_hst_stis_v-xx-cha_g230l-g430l-g750l_dr7_aspec.fits" --output "hlsp_ullyses_hst_stis_v-xx-cha_g230l-g430l-g750l_dr7_aspec.fits" --fail --create-dirs

In [ ]:
ls *.fits

\
Eleven datasets were just downloaded. They include:

Level 2
- hlsp_ullyses_hst_stis_sk-67d167_e140m_dr7_cspec.fits
- hlsp_ullyses_hst_cos_v-xx-cha_g130m_dr7_cspec.fits
- hlsp_ullyses_hst_cos_v-xx-cha_g160m_dr7_cspec.fits
- hlsp_ullyses_hst_stis_v-xx-cha_g230l_dr7_cspec.fits
- hlsp_ullyses_hst_stis_v-xx-cha_g430l_dr7_cspec.fits
- hlsp_ullyses_hst_stis_v-xx-cha_g750l_dr7_cspec.fits

Level 3
- hlsp_ullyses_fuse_fuv_sk-67d167_lwrs_dr7_aspec.fits
- hlsp_ullyses_hst_cos_v-xx-cha_g130m-g160m_dr7_aspec.fits
- hlsp_ullyses_hst_stis_v-xx-cha_g230l-g430l-g750l_dr7_aspec.fits

Level 4
- hlsp_ullyses_hst-fuse_fuse-stis_sk-67d167_uv_dr7_preview-spec.fits
- hlsp_ullyses_hst_cos-stis_v-xx-cha_uv-opt_dr7_preview-spec.fits

We will first go through the level 4 file, which is the data product with all instruments and settings abutted together. This file is intended to be used as a preview/visualization product only.

In [ ]:
targets = ['XX CHA', 'SK -67 167']

for targ in targets:
    ull_targ = match_aliases(targ).lower()
    print(ull_targ)
    # Create a variable to point to the file we just downloaded
    spec_filename = glob.glob(f'*{ull_targ}*_preview-spec.fits')[0]
    spec_file = Path(spec_filename)
    
    # Use Astropy.fits to open the fits file and create a header data unit (HDU)
    # More info on fits files and HDUs can be found at: https://docs.astropy.org/en/stable/io/fits/
    with fits.open(spec_file) as spec_hdu: 
        # Print out some information about the format and data in this HDU
        print(spec_hdu.info())
        
    print(f"**Filename components: {spec_filename.split('_')}")
    print()

\
From the filename, `hlsp_ullyses_hst_cos-stis_v-xx-cha_uv-opt_dr7_preview-spec.fits`, we can see:
- the telescope used is HST
- the instruments used are COS and STIS
- the target observed is V-XX-CHA
- the wavelength range is UV and Optical
- this file was created for Data Release 7

and from the filename, `hlsp_ullyses_hst-fuse_fuse-stis_sk-67d167_uv_dr7_preview-spec.fits`, we can see:
- the telescopes used are HST and FUSE
- the instruments used are STIS and FUSE ("FUSE" is also labeled as the instrument)
- the target observed is SK-67D167
- the wavelength range is UV
- this file was created for Data Release 7

Now we'll look at the science and provenance extensions.

#### Non-Variable Spectra Science Table

In the data extension for each spectrum, there are five columns: wavelength, flux, error, signal-to-noise ratio, and effective exposure time, all of the same size. The table extension headers also contain informative metadata, although we do not explore that fully here.

In [ ]:
# Print the columns in the science extension of the HDU - the science extension is index 1
spec_filename = glob.glob(f'*xx-cha*_preview-spec.fits')[0]
print(spec_filename)
with fits.open(spec_filename) as spec_hdu:
    print(spec_hdu[1].columns)

Now let's plot the flux, error, wavelength, and SNR for each level 4 (`preview-spec`) dataset. To get these values we use `.data` to access the data in the science extension of the HDU. To access the header of this extension instead, we would use `.header`

In [ ]:
# setting up a quick function to get the first extension data from the ULLYSES level 2, level 3, and level 4 products
def get_non_variable_data(filename):

    with fits.open(filename) as spec_hdu:
        # open all of the data arrays we will use
        wavelength = spec_hdu[1].data['WAVELENGTH'][0]
        flux = spec_hdu[1].data['FLUX'][0]
        snr = spec_hdu[1].data['SNR'][0]
        error = spec_hdu[1].data['ERROR'][0]
        grating = ' & '.join(np.unique(spec_hdu[2].data['DISPERSER'])) # unique gratings used from provenance extension

        # set the zeros to nans for better plotting; there will be a gap instead of a dip around a data quality flag
        flux[np.where(flux == 0.0)[0]] = "nan"
        error[np.where(error == 0.0)[0]] = "nan"
        snr[np.where(snr == 0.0)[0]] = "nan" 

    return wavelength, flux, error, snr, grating

In [ ]:
# Plot the level 4 preview-spec for each of the targets 
for targ in targets:
    # use the built in ULLYSES function to find the HLSP name for the target
    ull_targ = match_aliases(targ).lower()
    
    # Create a variable to point to the correct preview-spec file
    prevspec_filename = glob.glob(f'*{ull_targ}*_preview-spec.fits')[0]
    prevspec_file = Path(prevspec_filename)

    wavelength, flux, error, snr, gratings = get_non_variable_data(prevspec_file)    
        
    # Set up the plot
    fig = plt.figure()
    gs = fig.add_gridspec(3, hspace=0)
    axs = gs.subplots(sharex=True, sharey=False)
    axs[0].plot(wavelength, flux, color='k')
    axs[1].plot(wavelength, error, color='k')
    axs[2].plot(wavelength, snr, color='k')

    # add the labels & limits
    axs[0].set_xlim(wavelength.min(), wavelength.max())
    if ull_targ == 'v-xx-cha':
        axs[0].set_ylim(-1E-15, 5E-14)
        axs[1].set_ylim(-1E-15, 4E-15)
    
    plt.suptitle(f'{targ.upper()} preview-spec')
    axs[0].set_ylabel('Flux\n[ergs/s/cm^2/$\mathrm{\AA}$]')
    axs[1].set_ylabel('Error\n[ergs/s/cm^2/$\mathrm{\AA}$]')
    axs[2].set_ylabel('SNR')
    plt.xlabel('Wavelength [$\mathrm{\AA}$]')

Now that we know what the level 4 product (`preview-spec`) looks like, let's see how the combined level 2 products (`cspec`) compare.

In [ ]:
# Print the columns in the science extension of the HDU - the science extension is index 1
cspec_filename = glob.glob(f'*xx-cha*_cspec.fits')[0]
print(cspec_filename)
with fits.open(cspec_filename) as cspec_hdu:
    print(cspec_hdu.info())
    print()
    print(cspec_hdu[1].columns)

In [ ]:
# For each target, plot the level 4 preview-spec and the level 2 cspec products
for targ in targets:
    # use the built in ULLYSES function to find the HLSP name for the target
    ull_targ = match_aliases(targ).lower()
    
    # Create a variable to point to the correct preview-spec file
    prevspec_file = glob.glob(f'*{ull_targ}*_preview-spec.fits')[0]
    spec_file = Path(prevspec_file)

    # preview-spec data
    wavelength, flux, error, snr, gratings = get_non_variable_data(spec_file)  

    # Make an array to hold the level 2 filenames
    level2 = glob.glob(f'*{ull_targ}*cspec*')
    
    # Set up the plot - start with plotting the level 4 products
    fig = plt.figure()
    gs = fig.add_gridspec(3, hspace=0)
    axs = gs.subplots(sharex=True, sharey=False)

    axs[0].plot(wavelength, flux, color='k', label='preview-spec')
    axs[1].plot(wavelength, error, color='k')
    axs[2].plot(wavelength, snr, color='k')
    
    # Then use a for loop to add the level 2 products on top
    for prod in level2:
        # level 2 product data
        prod_wavelength, prod_flux, prod_error, prod_snr, prod_gratings = get_non_variable_data(prod) 

        axs[0].plot(prod_wavelength, prod_flux, alpha=0.8, label=prod_gratings)
        axs[1].plot(prod_wavelength, prod_error, alpha=0.8)
        axs[2].plot(prod_wavelength, prod_snr, alpha=0.8) 
    
    # Add all the labels & limits
    plt.suptitle(f'{targ.upper()} cspec')
    axs[0].set_ylabel('Flux\n[ergs/s/cm^2/$\mathrm{\AA}$]')
    axs[1].set_ylabel('Error\n[ergs/s/cm^2/$\mathrm{\AA}$]')
    axs[2].set_ylabel('SNR')
    plt.xlabel('Wavelength [$\mathrm{\AA}$]')
    axs[0].legend(ncol=len(level2)+1, loc='upper left')

    axs[0].set_xlim(wavelength.min(), wavelength.max())
    if ull_targ == 'v-xx-cha':
        axs[0].set_ylim(-1E-15, 5E-14)
        axs[1].set_ylim(-1E-15, 5E-15)
    else:
        # set the limit based on the preview-spec to avoid grating edges
        axs[0].set_ylim(np.nanmin(flux), np.nanmax(flux)) 
        axs[1].set_ylim(np.nanmin(error), np.nanmax(error)) 


In the above figures, we can see which combined `cspec` files are used in the abutted `preview-spec` files. A close look at the edges of each `cspec` file shows the decisions made by the abuttment logic for these gratings. This logic was determined to be the best for most targets, but the logic isn't perfect for every target, so the more indepth notebook [`ullyses-coadd.ipynb`](https://github.com/spacetelescope/ullyses/blob/main/notebooks/ullyses-coadd.ipynb) shows how to use the ULLYSES coadd and abut code to create your own products.


Finally, let's compare the level 3 product (`aspec`) with the level 4 product (`preview-spec`). The `aspec` file abuts all `cspec` files for gratings of similar resolution. Note that all FUSE spectra are level 3 products.

In [ ]:
# Print the columns in the science extension of the HDU - the science extension is index 1
aspec_filename = glob.glob(f'*xx-cha*_aspec.fits')[0]
print(aspec_filename)
with fits.open(aspec_filename) as aspec_hdu:
    print(aspec_hdu.info())
    print()
    print(aspec_hdu[1].columns)

In [ ]:
for targ in targets:
    # use the built in ULLYSES function to find the HLSP name for the target
    ull_targ = match_aliases(targ).lower()
    
    # Create a variable to point to the correct preview-spec file
    prevspec_filename = glob.glob(f'*{ull_targ}*_preview-spec.fits')[0]
    spec_file = Path(prevspec_filename)

    # Make an array to hold the level 3 filenames
    level3 = glob.glob(f'*{ull_targ}*aspec*')
    
    # Set up the plot - start with plotting the level 4 products
    fig = plt.figure()
    gs = fig.add_gridspec(3, hspace=0)
    axs = gs.subplots(sharex=True, sharey=False)

    # preview-spec data
    wavelength, flux, error, snr, gratings = get_non_variable_data(spec_file)  
    
    axs[0].plot(wavelength, flux, color='k', label='preview-spec')
    axs[1].plot(wavelength, error, color='k')
    axs[2].plot(wavelength, snr, color='k')
    
    # Then use a for loop to add the level 3 products on top
    for prod in level3:
        # level 3 product data
        prod_wavelength, prod_flux, prod_error, prod_snr, prod_gratings = get_non_variable_data(prod) 

        axs[0].plot(prod_wavelength, prod_flux, alpha=0.8, label=prod_gratings)
        axs[1].plot(prod_wavelength, prod_error, alpha=0.8)
        axs[2].plot(prod_wavelength, prod_snr, alpha=0.8) 
        
    
    # Add all the labels & limits
    plt.suptitle(f'{targ.upper()} aspec')
    axs[0].set_ylabel('Flux\n[ergs/s/cm^2/$\mathrm{\AA}$]')
    axs[1].set_ylabel('Error\n[ergs/s/cm^2/$\mathrm{\AA}$]')
    axs[2].set_ylabel('SNR')
    plt.xlabel('Wavelength [$\mathrm{\AA}$]')
    axs[0].legend(ncol=len(level3)+1, loc='upper left')

    axs[0].set_xlim(wavelength.min(), wavelength.max())
    if ull_targ == 'v-xx-cha':
        axs[0].set_ylim(-1E-15, 5E-14)
        axs[1].set_ylim(-1E-15, 5E-15)
    else:
        # set the limit based on the preview-spec to avoid grating edges
        axs[0].set_ylim(np.nanmin(flux), np.nanmax(flux)) 
        axs[1].set_ylim(np.nanmin(error), np.nanmax(error)) 


Like the above `cspec` figures, we can see here how the `aspec` files look compared to the `preview-spec` files. For XX Cha, there are two `aspec` files; one is made for the low resolution gratings, and one is made for the medium resolution gratings. For SK -67 167, there is only one `apec` file for the FUSE data. There is only one grating for the HST data (E140M), so the `cspec` file would be identical to the `aspec` file and therefore is not created again.

#### Non-Variable Provenance Table

For each spectrum that contributes to a given HLSP, select metadata is available in the provenance table. The fields in the table are metadata harvested from the headers of the contributing spectra. Notice that we already used the "DISPERSER" column in the plot labels above to see all contributing gratings in that product.

In [ ]:
# Print the columns in the provenance extension, which is index 2
print(spec_filename)
with fits.open(spec_filename) as spec_hdu:
    print(spec_hdu[2].columns)

In [ ]:
# We'll use Astropy Tables to print out a few of the columns listed above in a neat way
with fits.open(spec_filename) as spec_hdu:
    prov_table = Table(spec_hdu[2].data)

prov_table[['FILENAME', 
            'PROPOSID', 
            'INSTRUMENT', 
            'DETECTOR', 
            'DISPERSER', 
            'MJD_BEG', 
            'XPOSURE']]

In [ ]:
# To print all columns use:
prov_table

In [ ]:
# Or to print a single column use:
prov_table['FILENAME'].pprint()

The provenance extension can be used to determine what individual exposures were included in a combined (`cspec`) or abuted (`aspec` or `preview-spec`) product. For example, let's find only the COS G130M exposures that were included in the level 4 (`preview-spec`) file.

In [ ]:
# Use the provenance extension to find filenames for COS G130M exposures
prov_table[(prov_table["DISPERSER"] == "G130M") & (prov_table["INSTRUMENT"] == "COS")]

<a id="tss"></a>
### Timeseries Spectra

Spectroscopic time-series spectra (TSS) products are available for all T Tarui star (TTS) targets. These products are created from imaging data from the Las Cumbres Observatory (LCO) as well as HST spectroscopic data. The products available for each type of target and data are  summarized in the table below. Timeseries outputs are all level 5 data products.

| Target         | Data    | TSS Type                        |
| -------------- | ------- | ------------------------------- |
| Survey TTS     | LCO     | Exposure level                  |
| Survey TTS     | HST     | Exposure level                  |
| Monitoring TTS | LCO     | Exposure level                  |
| Monitoring TTS | HST     | Exposure and sub-exposure level |

#### FITS File Structure

Like the non-variable spectra, the TSS data and information is stored in two BINTABLE extensions:
- a Science extension for the combined product, and
- a matching Provenance extension to record attributes of each spectrum that contributed to the timeseries product

Next, we'll download an exposure-level TSS file for the star TW Hydra made from LCO data.

In [ ]:
# Use curl to download a TSS data product for the target TW Hya
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fv-tw-hya%2Fdr7%2Fhlsp_ullyses_lcogt_04m_v-tw-hya_uprime-v-iprime_dr7_tss.fits" --output "hlsp_ullyses_lcogt_04m_v-tw-hya_uprime-v-iprime_dr7_tss.fits" --fail --create-dirs

In [ ]:
!ls *tss*

In [ ]:
# Create a variable to point to the file we just downloaded, then open the file, and print the HDU info
exp_tss_filename = './hlsp_ullyses_lcogt_04m_v-tw-hya_uprime-v-iprime_dr7_tss.fits'
exp_tss_file = Path(exp_tss_filename)
exp_tss_hdu = fits.open(exp_tss_file)
print(exp_tss_hdu.info())

\
From the filename, `hlsp_ullyses_lcogt_04m_v-tw-hya_uprime-v-iprime_dr7_tss.fits`, we can see:
- the telescope used is LCOGT (the former name for LCO)
- the instrument used is the 0.4 meter
- the target observed is v-tw-hya (aka TW Hydra)
- the optical elements used are uprime, v, and iprime
- this file was created for Data Release 7

Now we'll look at the science and provenance extensions.

#### Time-series Spectra Science Table
The TSS products have slightly different table columns compared to non-variable spectra. The FLUX and ERROR arrays are 2D arrays with wavelength increasing X, and time increasing along Y. The wavelength values for each column of the 2D data are stored in the WAVELENGTH array, while the MJDSTART and MJDEND columns store the start and end times for each row of the FLUX and ERROR arrays.

In [ ]:
# Print the columns in the science extension
print(exp_tss_hdu[1].columns)

# Print the shape of the FLUX array
print('Number of time bins: {}'.format(exp_tss_hdu[1].data['FLUX'][0].shape[0]))
print('Number of spectral bins: {}'.format(exp_tss_hdu[1].data['FLUX'][0].shape[1]))

**A note about the LCO observations** \
LCO images are taken over four epochs, each consisting of daily observations over 10-day intervals. In addition, 15-minute cadence observations are also taken during the HST observations. 

For each daily observation, back-to-back images are taken in the applicable filters for the star type. Monitoring stars use SDSS u’ and i’ filters and the Bessel V filter, while survey stars only use the i’ and V filters.

Because the LCO imaging data is only observed in 2 or 3 different bandpasses, these data files will have only 2 or 3 wavelength bins, with the wavelength entries corresponding to the central wavelength of each filter.

In [ ]:
wavelength = exp_tss_hdu[1].data['WAVELENGTH'][0]
print(wavelength)

In [ ]:
flux = exp_tss_hdu[1].data['FLUX'][0]
print(flux)

In [ ]:
# separate the fluxes for each wavelength using a list comprehesion

flux_wave1 = np.array([x[0] for x in flux])   # this selects the first entry in each row, which is the flux at the first wavelength bin
flux_wave2 = np.array([x[1] for x in flux]) 
flux_wave3 = np.array([x[2] for x in flux])

In [ ]:
# Now, plot the flux vs time for each wavelength bin

# Make a variable 'time' for the MDJ start time
time = exp_tss_hdu[1].data['MJDSTART'][0]

# Filter out the data points that have 0 flux 
filter1 = np.where(flux_wave1 != 0.0)[0]
filter2 = np.where(flux_wave2 != 0.0)[0]
filter3 = np.where(flux_wave3 != 0.0)[0]

# Set up the plot
fig = plt.figure()

plt.scatter(time[filter1], flux_wave1[filter1], label=r'$\lambda$={}'.format(wavelength[0]))
plt.scatter(time[filter2], flux_wave2[filter2], label=r'$\lambda$={}'.format(wavelength[1]))
plt.scatter(time[filter3], flux_wave2[filter3], label=r'$\lambda$={}'.format(wavelength[2]))

# Set up the x ticks
mjdticks = np.linspace(time[0], time[-1], 7)
yyyymmdd_date = [Time(t, format='mjd').isot.split('T')[0] for t in mjdticks]    # the split after the 'T' selects the date only
plt.xticks(mjdticks, yyyymmdd_date)

plt.title('TW Hydra LCO Timeseries')
plt.ylabel('Flux\n[ergs/s/cm^2/$\mathrm{\AA}$]')
plt.xlabel('Exposure Date')
plt.legend()  
plt.show()

#### Time-series Spectra Provenance Table
Select metadata for each spectrum that contributes to the time-series product will populate a row in the provenance table. The fields in the table are metadata harvested from the headers of the contributing spectra. Some columns are only present for HST data or LCO data.

In [ ]:
# Print the columns in the provenance extension
print(exp_tss_hdu[2].columns)

In [ ]:
# We'll use Astropy Tables to make it print out a few of the columns listed above in a neat way
prov_table = Table(exp_tss_hdu[2].data)

# Print out the provenance data
prov_table[['FILENAME',
            'PROPOSID', 
            'INSTRUMENT', 
            'DETECTOR', 
            'FILTER', 
            'MJD_MID', 
            'XPOSURE']]

In [ ]:
# Be sure to close the HDU when we are finished exploring!
exp_tss_hdu.close()

#### Sub-exposure level TTS Data Product Example

Next let's check out the sub-exposure level TTS file for TW Hydra.

In [ ]:
# Use curl to download a TSS data product for the target
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fv-tw-hya%2Fdr7%2Fhlsp_ullyses_hst_cos_v-tw-hya_g160m_dr7_split-tss.fits" --output "hlsp_ullyses_hst_cos_v-tw-hya_g160m_dr7_split-tss.fits" --fail --create-dirs

In [ ]:
# Create a variable to point to the file we just downloaded, then open the file, and print the HDU info
subexp_tss_filename = './hlsp_ullyses_hst_cos_v-tw-hya_g160m_dr7_split-tss.fits'
subexp_tss_file = Path(subexp_tss_filename)
subexp_tss_hdu = fits.open(subexp_tss_file)
print(subexp_tss_hdu.info())

\
From the filename, `hlsp_ullyses_hst_cos_v-tw-hya_g160m_dr7_split-tss.fits`, we can see:
- the telescope used is HST
- the instrument used is COS
- the target observed is v-tw-hya (aka TW Hydra)
- the optical element is G160M
- this file was created for Data Release 7

Now we'll look at the science and provenance extensions.

In [ ]:
# Print the columns in the science extension
print(subexp_tss_hdu[1].columns)

# Print the shape of the FLUX array
print('Number of time bins: {}'.format(subexp_tss_hdu[1].data['FLUX'][0].shape[0]))
print('Number of spectral bins: {}'.format(subexp_tss_hdu[1].data['FLUX'][0].shape[1]))

In [ ]:
# Print the columns in the provenance extension
print(subexp_tss_hdu[2].columns)

In [ ]:
# We'll use Astropy Tables to make it print out a few of the columns listed above in a neat way

prov_table = Table(subexp_tss_hdu[2].data)

# Use pprint to print out the provenance data
prov_table[['FILENAME',
            'PROPOSID',
            'INSTRUMENT', 
            'DETECTOR', 
            'CENWAVE', 
            'XPOSURE',
            'MJD_MID']]

In [ ]:
fig = plt.figure()

# Make useful variable names for the pertinent columns in the file
flux = subexp_tss_hdu[1].data['flux'][0]
wavelength = subexp_tss_hdu[1].data['wavelength'][0]
mjdstart = [Time(t, format='mjd').isot for t in subexp_tss_hdu[1].data['mjdstart'][0]]
mjdend = [Time(t, format='mjd').isot for t in subexp_tss_hdu[1].data['mjdend'][0]]

# Since there are 720 time bins, we pick some random indices and plot the spectra corresponding to those times
indices = [0, 200, 400, 600, 719]
for i, ind in enumerate(indices):
    plt.plot(wavelength, flux[ind], label=f'index={ind}, {mjdstart[ind]}', alpha=0.7, lw=2)
plt.legend()
plt.xlabel('Wavelength [$\mathrm{\AA}$]')
plt.ylabel('Flux\n[ergs/s/cm^2/$\mathrm{\AA}$]')
plt.title('TW Hydra Spectral Time Series')
plt.xlim(1540, 1560)    # This zooms us in at CIV
plt.ylim(-0.1e-12, 2.2e-12)

Let's make one more plot of this data, looking over a shorter time scale now (~a day). We'll only change the `indices` list.

In [ ]:
fig = plt.figure()

# Make useful variable names for the pertinent columns in the file
flux = subexp_tss_hdu[1].data['flux'][0]
wavelength = subexp_tss_hdu[1].data['wavelength'][0]
mjdstart = [Time(t, format='mjd').isot for t in subexp_tss_hdu[1].data['mjdstart'][0]]
mjdend = [Time(t, format='mjd').isot for t in subexp_tss_hdu[1].data['mjdend'][0]]

indices = np.arange(190, 250, 10)
for i, ind in enumerate(indices):
    plt.plot(wavelength, flux[ind], label=f'index={ind}, {mjdstart[ind]}', alpha=0.7, lw=2)
plt.legend()
plt.xlabel('Wavelength [$\mathrm{\AA}$]')
plt.ylabel('Flux\n[ergs/s/cm^2/$\mathrm{\AA}$]')
plt.title('TW Hydra Spectral Time Series')
plt.xlim(1540, 1560)
plt.ylim(-0.1e-12, 2.2e-12)

In [ ]:
# Be sure to close the HDU when we are finished exploring!
subexp_tss_hdu.close()

<a id="drizzle"></a>
### WFC3 Drizzled Images

Drizzled data products are available for NGC3109 and Sextans A, which were taken before the spectroscopic observations in order to estimate exposure times. These are level 6 data products.

#### FITS File Structure

Drizzled and weight images are stored in two IMAGEHDU extensions, and PROVENANCE information stored in a BINTABLE extension:
- a Science extension for the drizzled science image
- a Weight extension for the weight map image
- a matching Provenance extension to record attributes of each image that contributed to the combined product

Last, we'll explore a drizzled image of Sextans A observed with WFC3 with the F336W filter.

In [ ]:
# Download the data product
!curl -L -X GET "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast%3AHLSP%2Fullyses%2Fsextans-a%2Fdr7%2Fhlsp_ullyses_hst_wfc3_sextans-a_f336w_dr7_drc.fits" --output "hlsp_ullyses_hst_wfc3_sextans-a_f336w_dr7_drc.fits" --fail --create-dirs

In [ ]:
# Create a variable to point to the file we just downloaded, open the file, and print the HDU info
drz_file = Path('./hlsp_ullyses_hst_wfc3_sextans-a_f336w_dr7_drc.fits')
drz_hdu = fits.open(drz_file)
print(drz_hdu.info())

\
From the filename, `hlsp_ullyses_hst_wfc3_sextans-a_f336w_dr7_drc.fits`, we can see:
- the telescope used is HST
- the instrument used is WFC3
- the target observed is Sextans A
- the optical element is F336W
- this file was created for Data Release 7

Now we'll look at the science, weight, and provenance extensions.

#### Drizzled Image Science Extension

This extension is an image array that contains the drizzled image data.

In [ ]:
# Print the size of the drizzled image that's in the science extension
print(drz_hdu[1].shape)

In [ ]:
# Plot the drizzled image
fig = plt.figure()
drz_data = drz_hdu[1].data
wcs = WCS(drz_hdu[1].header)  # this creates a WCS object from the image header coordinate information

plt.subplot(projection=wcs)
plt.imshow(drz_data, vmin=0, vmax=0.015)

plt.title('Sextans A - F336W')
plt.grid(color='white', ls=':', alpha=0.7)
plt.xlabel('Right Ascension')
plt.ylabel('Declination')
plt.show()

#### Drizzled Image Weight Map Extension

This extension is another image array that contains the weight map data. The weight image contains information about bad pixels in the image, in that bad pixels result in lower weight values.

In [ ]:
# Print the size of the weight map, which is the second extension. This should be the same as the drizzled image
print(drz_hdu[2].shape)

In [ ]:
# Plot the weight map
fig = plt.figure()
wgt_data = drz_hdu[2].data
wcs = WCS(drz_hdu[2].header)

plt.subplot(projection=wcs)
plt.imshow(wgt_data, vmin=0, vmax=900)

plt.title('Sextans A - F336W - weight map')
plt.grid(color='white', ls=':')
plt.xlabel('Right Ascension')
plt.ylabel('Declination')
plt.show()

#### Drizzled Image Provenance Table

Select metadata for each image that contributes to the drizzled image in will populate a row in the provenance table. The fields in the  table are metadata harvested from the headers of the contributing images.

In [ ]:
# Print the columns in the provenance extension
print(drz_hdu[3].columns)

In [ ]:
# We'll again use Astropy Tables to make it print out a few of the columns listed above in a neat way

prov_table = Table(drz_hdu[3].data)

prov_table[['FILENAME', 
            'PROPOSID',
            'INSTRUMENT',
            'DETECTOR', 
            'FILTER',
            'APERTURE',
            'XPOSURE']]

In [ ]:
# Close the DRZ HDU!
drz_hdu.close()

***

## Additional Resources

- [ULLYSES](https://ullyses.stsci.edu)
- [MAST API](https://mast.stsci.edu/api/v0/index.html)

## About this Notebook
For support, contact us at the [ULLYSES Helpdesk](https://stsci.service-now.com/hst?id=sc_cat_item&sys_id=a3b8ec5edbb7985033b55dd5ce961990&sysparm_category=ac85189bdb4683c033b55dd5ce96199c).

**Authors:**  Elaine Frazer, Rachel Plesha \
**Updated On:** March 11, 2024

## Citations
* See the [ULLYSES website](https://ullyses.stsci.edu/ullyses-cite.html) for citation guidelines.


***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 